In [1]:
import warnings 
warnings.filterwarnings('ignore')
import time 
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import GradientBoostingRegressor, BaggingRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
import lightgbm as lgb
import tensorflow as tf 

In [2]:
def get_score(model, X, y_true):
    y_pred = model.predict(X)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def mean_absolute_precision_error(y_pred, y_true):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Data gathering

In [3]:
data      = pd.read_csv('/Users/mohsenkiskani/.kaggle/competitions/ubaar-competition/train.csv')
test_data = pd.read_csv('/Users/mohsenkiskani/.kaggle/competitions/ubaar-competition/test.csv')

data      = data.dropna(axis = 0)

test_data.loc[12577, 'distanceKM']      = 52
test_data.loc[12577, 'taxiDurationMin'] = 50
test_data.loc[13853, 'distanceKM']      = 500
test_data.loc[13853, 'taxiDurationMin'] = 380

all_data = pd.concat((data, test_data)) 
all_data['source']           = all_data['sourceLatitude']*all_data['sourceLongitude']
all_data['destination']      = all_data['destinationLatitude']*all_data['destinationLongitude']

min_price = min(all_data['price'])

ntrain = data.shape[0]
ntest  = test_data.shape[0]

categorical_vars = ['date', 'SourceState', 'destinationState', 'vehicleType', 'vehicleOption']

dummies_data = pd.get_dummies(all_data[categorical_vars])
all_data[dummies_data.columns] = dummies_data[dummies_data.columns]
all_data.drop(categorical_vars, axis=1, inplace=True)

train    = all_data[:ntrain]
test     = all_data[ntrain:]

train_1, train_2 = train_test_split(train, test_size=0.5)

X = train.drop(['ID','price'],axis=1)
y = train.price

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.head()

,destinationLatitude,destinationLongitude,distanceKM,sourceLatitude,sourceLongitude,taxiDurationMin,weight,source,destination,SourceState_آذربایجان شرقی,...,vehicleType_treili,vehicleOption_bari,vehicleOption_hichkodam,vehicleOption_kafi,vehicleOption_kompressi,vehicleOption_labehdar,vehicleOption_mosaghaf_chadori,vehicleOption_mosaghaf_felezi,vehicleOption_transit_chadori,vehicleOption_yakhchali
39085,36.473089,52.349822,184.0,35.700109,51.399743,199.0,21.00,1834.976428,1909.359717,0,...,1,0,0,1,0,0,0,0,0,0
30892,35.704176,51.400280,331.0,37.275731,49.584392,254.0,1.67,1848.294458,1835.204644,0,...,0,0,0,0,0,0,1,0,0,0
45277,35.699924,51.396715,447.0,32.665899,51.663805,285.0,19.00,1687.644636,1834.858819,0,...,1,0,0,0,0,0,0,0,1,0
16398,30.199563,53.182966,809.0,35.699078,51.401589,525.0,4.00,1834.989335,1606.102332,0,...,0,0,0,0,0,0,1,0,0,0
13653,27.180941,56.277756,1144.0,34.643252,50.877469,750.0,2.00,1762.560980,1529.682365,0,...,0,0,0,0,0,0,1,0,0,0


# Initial models 

In [ ]:
start_time = time.time()
GBoost_1 = GradientBoostingRegressor(n_estimators=3200, learning_rate=0.05,
                                     max_depth=10, max_features='sqrt',
                                     min_samples_leaf=15, min_samples_split=10, loss='huber')

GBoost_2 = GradientBoostingRegressor(n_estimators=3200, learning_rate=0.05,
                                     max_depth=10, max_features='sqrt',
                                     min_samples_leaf=15, min_samples_split=10, loss='huber')

GBoost_1.fit(train_1.drop(['ID','price'],axis=1), train_1.price)
GBoost_2.fit(train_2.drop(['ID','price'],axis=1), train_2.price)

train_1['y_gboost'] = GBoost_2.predict(train_1.drop(['ID','price'],axis=1))
train_2['y_gboost'] = GBoost_1.predict(train_2.drop(['ID','price'],axis=1))

train_gboost = pd.concat([train_1, train_2])
print( '%.2f' % float((time.time() - start_time)/60 )  + ' mins')
train_gboost.head()

# 5.85 mins

In [ ]:
train_gboost_1, train_gboost_2 = train_test_split(train_gboost, test_size=0.5)

In [ ]:
start_time = time.time()
xgb_1 = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                         learning_rate=0.05, max_depth=10, 
                         min_child_weight=1.7817, n_estimators=2200,
                         reg_alpha=0.4640, reg_lambda=0.8571,
                         subsample=0.5213, silent=1, nthread = -1)

xgb_2 = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                         learning_rate=0.05, max_depth=10, 
                         min_child_weight=1.7817, n_estimators=2200,
                         reg_alpha=0.4640, reg_lambda=0.8571,
                         subsample=0.5213, silent=1, nthread = -1)

xgb_1.fit(train_gboost_1.drop(['ID','price','y_gboost'],axis=1), train_gboost_1.price)
xgb_2.fit(train_gboost_2.drop(['ID','price','y_gboost'],axis=1), train_gboost_2.price)

train_gboost_1['y_xgb'] = xgb_2.predict(train_gboost_1.drop(['ID','price','y_gboost'],axis=1))
train_gboost_2['y_xgb'] = xgb_1.predict(train_gboost_2.drop(['ID','price','y_gboost'],axis=1))

train_xgb = pd.concat([train_gboost_1, train_gboost_2])
print( '%.2f' % float((time.time() - start_time)/60 )  + ' mins')
train_xgb.head()

# 8.20 mins

In [ ]:
train_xgb_1, train_xgb_2 = train_test_split(train_xgb, test_size=0.5)

In [ ]:
start_time = time.time()

bag_1 = BaggingRegressor(n_estimators=1000, max_samples=1.0, max_features=1.0, verbose=1)
bag_2 = BaggingRegressor(n_estimators=1000, max_samples=1.0, max_features=1.0, verbose=1)

bag_1.fit(train_xgb_1.drop(['ID','price','y_gboost','y_xgb'],axis=1), train_xgb_1.price)
bag_2.fit(train_xgb_2.drop(['ID','price','y_gboost','y_xgb'],axis=1), train_xgb_2.price)

train_xgb_1['y_bag'] = bag_2.predict(train_xgb_1.drop(['ID','price','y_gboost','y_xgb'],axis=1))
train_xgb_2['y_bag'] = bag_1.predict(train_xgb_2.drop(['ID','price','y_gboost','y_xgb'],axis=1))

train_bag = pd.concat([train_xgb_1, train_xgb_2])
print( '%.2f' % float((time.time() - start_time)/60 )  + ' mins' )
train_bag.head()

# 10.23 mins

In [ ]:
train_bag.to_pickle('dataFrames/train_OneHotEncoding.pkl')

In [ ]:
train_bag = pd.read_pickle('dataFrames/train_OneHotEncoding.pkl')
train_bag_1, train_bag_2 = train_test_split(train_bag, test_size=0.5)

In [ ]:
start_time = time.time()

knn_1 = KNeighborsClassifier(2)
knn_2 = KNeighborsClassifier(2)

knn_1.fit(train_bag_1.drop(['ID','price','y_gboost','y_xgb', 'y_bag'],axis=1), train_bag_1.price)
knn_2.fit(train_bag_2.drop(['ID','price','y_gboost','y_xgb', 'y_bag'],axis=1), train_bag_2.price)

train_bag_1['y_knn'] = knn_2.predict(train_bag_1.drop(['ID','price','y_gboost','y_xgb', 'y_bag'],axis=1))
train_bag_2['y_knn'] = knn_1.predict(train_bag_2.drop(['ID','price','y_gboost','y_xgb', 'y_bag'],axis=1))

train_knn = pd.concat([train_bag_1, train_bag_2])
print( '%.2f' % float((time.time() - start_time)/60 )  + ' mins' )

train_knn.to_pickle('dataFrames/train_OneHotEncoding.pkl')
train_knn.head()

# 0.11 mins

In [ ]:
train_knn = pd.read_pickle('dataFrames/train_OneHotEncoding.pkl')
train_knn_1, train_knn_2 = train_test_split(train_knn, test_size=0.5)

In [ ]:
start_time = time.time()

dec_1 = DecisionTreeRegressor(max_depth=10)
dec_2 = DecisionTreeRegressor(max_depth=10)

dec_1.fit(train_knn_1.drop(['ID','price','y_gboost','y_xgb', 'y_bag', 'y_knn'],axis=1), train_knn_1.price)
dec_2.fit(train_knn_2.drop(['ID','price','y_gboost','y_xgb', 'y_bag', 'y_knn'],axis=1), train_knn_2.price)

train_knn_1['y_dec'] = dec_2.predict(train_knn_1.drop(['ID','price','y_gboost','y_xgb', 'y_bag', 'y_knn'],axis=1))
train_knn_2['y_dec'] = dec_1.predict(train_knn_2.drop(['ID','price','y_gboost','y_xgb', 'y_bag', 'y_knn'],axis=1))

train_dec = pd.concat([train_knn_1, train_knn_2])
print( '%.2f' % float((time.time() - start_time)/60 )  + ' mins' )

train_dec.to_pickle('dataFrames/train_OneHotEncoding.pkl')
train_dec.head()

# 0.01 mins

In [ ]:
train_dec = pd.read_pickle('dataFrames/train_OneHotEncoding.pkl')
train_dec_1, train_dec_2 = train_test_split(train_dec, test_size=0.5)

In [ ]:
start_time = time.time()

lgb_1 = lgb.LGBMRegressor(objective='regression',num_leaves=15,
                          learning_rate=0.05, n_estimators=15000,
                          max_bin = 1000, bagging_fraction = 0.6,
                          bagging_freq = 5, feature_fraction = 0.25,
                          feature_fraction_seed=9, bagging_seed=20,
                          min_data_in_leaf = 11, min_sum_hessian_in_leaf = 11)

lgb_2 = lgb.LGBMRegressor(objective='regression',num_leaves=15,
                          learning_rate=0.05, n_estimators=15000,
                          max_bin = 1000, bagging_fraction = 0.6,
                          bagging_freq = 5, feature_fraction = 0.25,
                          feature_fraction_seed=9, bagging_seed=20,
                          min_data_in_leaf = 11, min_sum_hessian_in_leaf = 11)

lgb_1.fit(train_dec_1.drop(['ID','price','y_gboost','y_xgb', 'y_bag', 'y_knn', 'y_dec'],axis=1), train_dec_1.price)
lgb_2.fit(train_dec_2.drop(['ID','price','y_gboost','y_xgb', 'y_bag', 'y_knn', 'y_dec'],axis=1), train_dec_2.price)

train_dec_1['y_lgb'] = lgb_2.predict(train_dec_1.drop(['ID','price','y_gboost','y_xgb', 'y_bag', 'y_knn', 'y_dec'],axis=1))
train_dec_2['y_lgb'] = lgb_1.predict(train_dec_2.drop(['ID','price','y_gboost','y_xgb', 'y_bag', 'y_knn', 'y_dec'],axis=1))

train_lgb = pd.concat([train_dec_1, train_dec_2])
print( '%.2f' % float((time.time() - start_time)/60 ) + ' mins' )

train_lgb.to_pickle('dataFrames/train_OneHotEncoding.pkl')
train_lgb.head()

# 2.26 mins

# Test dataset augmentation

In [ ]:
start_time = time.time()
GBoost = GradientBoostingRegressor(n_estimators=2200, learning_rate=0.05,
                                   max_depth=10, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, loss='huber')

xgb_model = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=10, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1, nthread = -1)

bag   = BaggingRegressor(n_estimators=1000, max_samples=1.0, max_features=1.0, verbose=1)
knn   = KNeighborsClassifier(2)
dec   = DecisionTreeRegressor(max_depth=10)
lgb_m = lgb.LGBMRegressor(objective='regression',num_leaves=15,
                       learning_rate=0.05, n_estimators=15000,
                       max_bin = 1000, bagging_fraction = 0.6,
                       bagging_freq = 5, feature_fraction = 0.25,
                       feature_fraction_seed=9, bagging_seed=20,
                       min_data_in_leaf = 11, min_sum_hessian_in_leaf = 11)

GBoost.fit(train.drop(['ID','price'],axis=1), train.price)
xgb_model.fit(train.drop(['ID','price'],axis=1), train.price)
bag.fit(train.drop(['ID','price'],axis=1), train.price)
knn.fit(train.drop(['ID','price'],axis=1), train.price)
dec.fit(train.drop(['ID','price'],axis=1), train.price)
lgb_m.fit(train.drop(['ID','price'],axis=1), train.price)



test['y_gboost'] = GBoost.predict(test.drop(['ID','price'],axis=1))
test['y_xgb']    = xgb_model.predict(test.drop(['ID','price','y_gboost'],axis=1))
test['y_bag']    = bag.predict(test.drop(['ID','price','y_gboost','y_xgb'],axis=1))
test['y_knn']    = knn.predict(test.drop(['ID','price','y_gboost','y_xgb', 'y_bag'],axis=1))
test['y_dec']    = dec.predict(test.drop(['ID','price','y_gboost','y_xgb', 'y_bag', 'y_knn'],axis=1))
test['y_lgb']    = lgb_m.predict(test.drop(['ID','price','y_gboost','y_xgb', 'y_bag', 'y_knn', 'y_dec'],axis=1))

print( '%.2f' % float((time.time() - start_time)/60 ), "mins" )


test.to_pickle('dataFrames/test_OneHotEncoding.pkl')
test.head()

# 25.04 mins

# Loading augmented data

In [4]:
test  = pd.read_pickle('dataFrames/test_OneHotEncoding.pkl')
train = pd.read_pickle('dataFrames/train_OneHotEncoding.pkl')

In [5]:
test.head()

,ID,destinationLatitude,destinationLongitude,distanceKM,price,sourceLatitude,sourceLongitude,taxiDurationMin,weight,source,...,vehicleOption_mosaghaf_chadori,vehicleOption_mosaghaf_felezi,vehicleOption_transit_chadori,vehicleOption_yakhchali,y_gboost,y_xgb,y_bag,y_knn,y_dec,y_lgb
0,10010571124,35.579635,53.384990,684.0,NaN,36.297213,59.607970,446.0,2.33,2163.603184,...,1,0,0,0,2.516722e+06,2.403949e+06,2603010.0,2200000.0,3.997050e+06,2.469241e+06
1,10031704713,29.605761,52.533588,931.0,NaN,35.704695,51.405194,614.0,19.14,1835.406773,...,0,0,0,0,1.003544e+07,1.027378e+07,9555599.0,8767000.0,1.022329e+07,1.077080e+07
2,10040911649,36.299593,59.612010,1469.0,NaN,26.948490,55.583875,1009.0,22.00,1497.901500,...,0,0,0,0,2.128578e+07,1.877921e+07,17803149.0,8687000.0,1.300000e+07,2.163461e+07
3,10047106840,35.248298,58.457567,745.0,NaN,35.339066,52.075970,496.0,2.50,1840.316141,...,0,1,0,0,2.052077e+06,1.795059e+06,2396458.0,2170000.0,2.772043e+06,1.919848e+06
4,10050126039,34.636832,50.874888,281.0,NaN,35.579577,53.394403,181.0,23.50,1899.750273,...,0,0,0,0,5.822093e+06,5.748156e+06,7099862.0,1900000.0,5.984402e+06,5.194802e+06


# TensorFlow combination  

In [46]:
BATCH_SIZE          = 128
TRAIN_EPOCHS        = 1200

HIDDEN_LAYER_1_SIZE = 512
HIDDEN_LAYER_2_SIZE = 512
HIDDEN_LAYER_3_SIZE = 16
lr                  = 5e-5

In [47]:
y_gboost_feat = tf.feature_column.numeric_column("y_gboost")
y_xgb_feat    = tf.feature_column.numeric_column("y_xgb")
y_bag_feat    = tf.feature_column.numeric_column("y_bag")
y_knn_feat    = tf.feature_column.numeric_column("y_knn")
y_dec_feat    = tf.feature_column.numeric_column("y_dec")
y_lgb_feat    = tf.feature_column.numeric_column("y_lgb")

source_lat_feat         = tf.feature_column.numeric_column("sourceLatitude") 
source_long_feat        = tf.feature_column.numeric_column("sourceLongitude") 
destin_lat_feat         = tf.feature_column.numeric_column("destinationLatitude") 
destin_long_feat        = tf.feature_column.numeric_column("destinationLongitude") 

distance_feat = tf.feature_column.numeric_column("distanceKM")
taximin_feat  = tf.feature_column.numeric_column("taxiDurationMin")
weight_feat   = tf.feature_column.numeric_column("weight")

source_feat   = tf.feature_column.numeric_column("source")
destin_feat   = tf.feature_column.numeric_column("destination")

feature_columns = {y_gboost_feat, y_xgb_feat, y_bag_feat, y_knn_feat, y_dec_feat, y_lgb_feat, 
                   source_lat_feat, source_long_feat , destin_lat_feat, destin_long_feat, 
                   distance_feat, taximin_feat, weight_feat, source_feat, destin_feat}

#feature_columns = {y_gboost_feat, y_xgb_feat, y_bag_feat, y_knn_feat, y_dec_feat, y_lgb_feat}

In [48]:
def make_model(features, labels, mode, params, config):
    input_layer = tf.feature_column.input_layer(features=features, 
                                                feature_columns=feature_columns)
    
    global_step = tf.train.get_or_create_global_step()

    x = tf.layers.dense(inputs=input_layer,
                        units=HIDDEN_LAYER_1_SIZE,
                        activation=tf.nn.relu,
                        name="first_fully_connected_layer")

    x = tf.layers.dropout(inputs=x,name="first_dropout")

    x = tf.layers.dense(inputs=x,
                        units=HIDDEN_LAYER_2_SIZE,
                        activation=tf.nn.relu,
                        name="second_fully_connected_layer")

    x = tf.layers.dense(inputs=x,
                        units=HIDDEN_LAYER_3_SIZE,
                        activation=tf.nn.relu,
                        name="third_fully_connected_layer")

    predictions = tf.contrib.layers.fully_connected(inputs=x, num_outputs=1)

    if mode == tf.estimator.ModeKeys.PREDICT :
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    elif mode == tf.estimator.ModeKeys.EVAL:
        loss  = tf.reduce_mean(tf.abs(tf.divide(predictions-labels,labels))) 
        return tf.estimator.EstimatorSpec(mode=mode,
                                          predictions=predictions,
                                          loss=loss)
    else:
        #loss = tf.losses.absolute_difference(labels=labels,
        #                                    predictions=predictions)
        loss  = tf.reduce_mean(tf.abs(tf.divide(predictions-labels,labels))) 
        tf.summary.scalar("Loss", loss)
        optimizer = tf.train.AdamOptimizer(learning_rate=params.learning_rate)
        train_op = optimizer.minimize(loss, 
                                      global_step=global_step)
        return tf.estimator.EstimatorSpec(mode=mode, 
                                          predictions=predictions,
                                          loss=loss, 
                                          train_op=train_op)

In [49]:
def input_fn(df, pred = False):
        
    useful_fueatures = [
        np.array(df["y_gboost"].values, dtype=np.float32),
        np.array(df["y_xgb"].values, dtype=np.float32),
        np.array(df["y_bag"].values, dtype=np.float32),
        np.array(df["y_knn"].values, dtype=np.float32),
        np.array(df["y_dec"].values, dtype=np.float32),
        np.array(df["y_lgb"].values, dtype=np.float32),
        np.array(df["sourceLatitude"].values, dtype=np.float32),
        np.array(df["sourceLongitude"].values, dtype=np.float32),
        np.array(df["destinationLatitude"].values, dtype=np.float32),
        np.array(df["destinationLongitude"].values, dtype=np.float32),
        np.array(df["distanceKM"].values, dtype=np.float32),
        np.array(df["taxiDurationMin"].values, dtype=np.float32),
        np.array(df["weight"].values, dtype=np.float32),
        #np.array(df["date_ids"].values, dtype=np.int32),
        #np.array(df["SourceState_ids"].values, dtype=np.int32),
        #np.array(df["destinationState_ids"].values, dtype=np.int32),
        #np.array(df["vehicleType_ids"].values, dtype=np.int32),
        #np.array(df["vehicleOption_ids"].values, dtype=np.int32),
        np.array(df["source"].values, dtype=np.float32),
        np.array(df["destination"].values, dtype=np.float32),
        #np.array(df["destination_tuple_ids"].values, dtype=np.int32),
        #np.array(df["source_tuple_ids"].values, dtype=np.int32)
    ]

    if pred: 
        train_number = 1
        batch_number = 1
    else:
        useful_fueatures.append(np.array(df["price"].values, dtype=np.float32))
        train_number = TRAIN_EPOCHS
        batch_number = BATCH_SIZE
        
    A = tf.train.slice_input_producer(
        tensor_list=useful_fueatures,
        num_epochs=train_number,
        shuffle= not pred,
        capacity=BATCH_SIZE * 5
    )
    
    y_gboost              = A[0]
    y_xgb                 = A[1]
    y_bag                 = A[2]
    y_knn                 = A[3]
    y_dec                 = A[4]
    y_lgb                 = A[5]
    sourceLatitude        = A[6]
    sourceLongitude       = A[7]
    destinationLatitude   = A[8]
    destinationLongitude  = A[9]
    distanceKM            = A[10]
    taxiDurationMin       = A[11] 
    weight                = A[12]
    #date_ids              = A[13]
    #SourceState_ids       = A[14]
    #destinationState_ids  = A[15]
    #vehicleType_ids       = A[16]
    #vehicleOption_ids     = A[17]
    source                = A[13]
    destination           = A[14] 
    #destination_tuple_ids = A[20] 
    #source_tuple_ids      = A[21] 
    
    # Created a dict out of sliced input producers
    dataset_dict = dict(
        y_gboost=y_gboost,
        y_xgb=y_xgb,
        y_bag=y_bag,
        y_knn=y_knn,
        y_dec=y_dec,
        y_lgb=y_lgb,
        sourceLatitude=sourceLatitude,
        sourceLongitude=sourceLongitude,
        destinationLatitude=destinationLatitude,
        destinationLongitude=destinationLongitude, 
        distanceKM=distanceKM,
        taxiDurationMin=taxiDurationMin,
        weight=weight,
        #date_ids=date_ids,
        #SourceState_ids=SourceState_ids,
        #destinationState_ids=destinationState_ids,
        #vehicleType_ids=vehicleType_ids,
        #vehicleOption_ids=vehicleOption_ids,
        source=source, 
        destination=destination,
        #destination_tuple_ids=destination_tuple_ids,
        #source_tuple_ids=source_tuple_ids,
    )

    if not pred:
        dataset_dict['labels'] = A[15]
            
    batch_dict = tf.train.batch(
        dataset_dict,
        batch_number,
   )

    if pred == False:
        batch_labels = batch_dict.pop('labels')
        return batch_dict, tf.reshape(batch_labels, [-1, 1]) 
    else:
        return batch_dict 

In [50]:
X_train, X_val = train_test_split(train, test_size=0.2, random_state=42)
X_train.head()

,ID,destinationLatitude,destinationLongitude,distanceKM,price,sourceLatitude,sourceLongitude,taxiDurationMin,weight,source,...,vehicleOption_mosaghaf_chadori,vehicleOption_mosaghaf_felezi,vehicleOption_transit_chadori,vehicleOption_yakhchali,y_gboost,y_xgb,y_bag,y_knn,y_dec,y_lgb
40689,10602550191,34.319566,47.078555,1172.0,19000000.0,27.474105,52.603738,859.0,22.00,1445.240621,...,0,0,0,0,2.001315e+07,20377512.00,20693972.0,4290000.0,1.883525e+07,2.214366e+07
28663,59022077023,35.699056,51.402792,911.0,8000000.0,29.617603,51.652078,661.0,5.35,1529.810740,...,0,0,0,0,5.602654e+06,7494018.50,4657211.0,6000000.0,4.240625e+06,7.224867e+06
19042,74752147720,27.182853,56.273862,980.0,10338000.0,32.801625,51.689466,667.0,23.61,1695.498480,...,0,0,0,0,9.197673e+06,9348608.00,9439628.0,8305000.0,9.854376e+06,1.029661e+07
21837,76223312658,32.673139,51.670482,200.0,2320000.0,34.136752,50.566116,142.0,11.30,1726.162961,...,0,0,0,0,2.536652e+06,2578700.75,2509670.0,2320000.0,2.347938e+06,3.293571e+06
35006,74609642925,35.699332,51.395552,935.0,18000000.0,30.435378,49.111278,659.0,22.00,1494.720310,...,0,0,0,0,1.737643e+07,17874336.00,17712615.0,16650000.0,1.878682e+07,1.692893e+07


In [51]:
hparams = tf.contrib.training.HParams(learning_rate=lr)
estimator_val = tf.estimator.Estimator(model_fn=make_model, params=hparams)
estimator_val.train(input_fn=lambda: input_fn(X_train), steps=600)#TRAIN_EPOCHS)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/hy/j_c72d1x72g_rr58tgrlh3b40000gn/T/tmpo0dro1xw', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c304ad550>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/hy

In [52]:
predictions_val   = list(estimator_val.predict(input_fn = lambda: input_fn(X_val, pred=True)))
y_preds_val       = [int(x) for x in predictions_val]
mean_absolute_precision_error(y_preds_val, X_val.price)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/hy/j_c72d1x72g_rr58tgrlh3b40000gn/T/tmpo0dro1xw/model.ckpt-600
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


16.70474557722731

In [55]:
estimator = tf.estimator.Estimator(model_fn=make_model, params=hparams)
estimator.train(input_fn=lambda: input_fn(train), steps=TRAIN_EPOCHS)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/hy/j_c72d1x72g_rr58tgrlh3b40000gn/T/tmp3mq22du1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a291c0748>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/hy

In [ ]:
predictions   = list(estimator.predict(input_fn = lambda: input_fn(test, pred=True)))
y_preds_test   = [int(x) for x in predictions]

# Save to File

In [ ]:
filename = "/Users/mohsenkiskani/Downloads/Ubaar/submissions/submission32.csv"
with open(filename,"w+") as outputfile:
    outputfile.write("ID,price\n")
    for i in range(len(y_preds_test)):
        outputfile.write(str(test_data.ID[i])+","+str(int(np.ceil(y_preds_test[i])))+"\n")

##### Submission 19 with loss of 15.9 